In [1]:
import networkx as nx
import networkx.algorithms.community as nxcom
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (15, 10)})
# get reproducible results
import random
from numpy import random as nprand
random.seed(123)
nprand.seed(123)

In [3]:
G = nx.read_edgelist(path = "../test/clusterPair_lik_gene_edges.csv", data=(('weight',float),), create_using=nx.Graph())
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 4292
Number of edges: 204042
Average degree:  95.0801


In [4]:
# read communities 
comm_file = open('../test/comm2genes.csv', 'r')
comm2genes = {}
for line in comm_file:    
    line = line.strip('\n').split('\t')    
    line = list(x for x in line if x != 'NA')
    (key, val) = [line[0], line[1:]]
    comm2genes[key] = val
    
comm_file.close()    

In [7]:
# add all other genes to unassigned group
comm2genes['unassigned'] = G.nodes() - set([item for sublist in comm2genes.values()  for item in sublist])

In [8]:
for c, v_c in comm2genes.items():
    print(c)

gc21
gc74
gc15
gc9
gc93
gc40
gc49
gc114
gc36
gc119
unassigned


In [10]:
def set_node_community(G, communities):
    '''Add community to node attributes'''
    for c, v_c in communities.items():
        for v in v_c:
            # Add 1 to save 0 for external edges
            G.nodes[v]['community'] = c
            
def set_edge_community(G, communities):
    '''Find internal edges and add their community to their attributes'''
    for v, w, in G.edges:
        if G.nodes[v]['community'] == G.nodes[w]['community'] and G.nodes[v]['community'] in communities:
            # Internal edge, mark with community
            G.edges[v, w]['community'] = G.nodes[v]['community']
        else:
            # External edge, mark as 0
            G.edges[v, w]['community'] = 0


In [11]:
comm2genes.keys()

dict_keys(['gc21', 'gc74', 'gc15', 'gc9', 'gc93', 'gc40', 'gc49', 'gc114', 'gc36', 'gc119', 'unassigned'])

In [12]:
# use numerical keys 
keys = {'gc21':1, 'gc74':2, 'gc15':3, 'gc9':4, 'gc93':5, 'gc40':6, 'gc49':7, 'gc114':8, 'gc36':9, 'gc119':10, 'unassigned':11}
comm2genes = dict((keys[key], value) for (key, value) in comm2genes.items())

In [13]:
# Set node and edge communities
set_node_community(G, comm2genes)

In [14]:
import numpy as np 
set_edge_community(G, np.arange(1,11))

In [15]:
communities = np.arange(1,11)

In [16]:
colormap = {1:'#0c343d', 2:'#feffb3', 3:'#5b0f00', 4:'#fa8174', 5:'#81b1d2', 6:'#fdb462', 7:'#b3de69', 8:'#bc82bd', 9:'#217e13', 10:'#ffed6f', 11:[0, 0, 0]}

In [17]:
node_color = [colormap[G.nodes[v]['community']] for v in G.nodes]

In [18]:
# Set community color for edges between members of the same community (internal) and intra-community edges (external)
external = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] == 0]
internal = [(v, w) for v, w in G.edges if G.edges[v, w]['community'] > 0]
internal_color = ['black' for e in internal]

In [19]:
pos = nx.spring_layout(G, k = 0.1, seed= 19901012)

In [20]:
import pickle
def pklDump(obj, file):
    objF = open(file, 'wb')
    pickle.dump(obj, objF)
    objF.close()

def pklLoad(file): 
    objF = open(file, 'rb')
    obj = pickle.load(objF)
    objF.close()
    return obj

In [21]:
pklDump(pos, "nw_springLayout.pkl")

In [166]:
#pos = pklLoad("nw_springLayout.pkl")

In [23]:
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (10, 8)})
plt.style.use('dark_background')
# Draw nodes
# Draw internal edges
nx.draw_networkx_edges(
    G,
    pos=pos,    
    edgelist=internal,
    alpha = 0.2,
    with_labels=False)
    
# Draw external edges    
nx.draw_networkx_edges(
    G,
    pos=pos,    
    edgelist=external,
    edge_color="silver",
    alpha = 0.2,
    with_labels=False)

# Draw nodes
for i in list(comm2genes.keys())[:-1]: 
    nodes_i = comm2genes[i]
    nx.draw_networkx_nodes(
        G,
        pos=pos,
        nodelist = nodes_i,
        node_size = 50,
        node_color=colormap[i],
        label=list(keys.keys())[list(keys.values()).index(i)],
        alpha=0.6, 
        with_labels=False)
    
plt.legend(numpoints = 1) #Instead of numpoints = 1, use scatterpoints = 1
plt.savefig("communities_labeled.png", dpi=300)

In [25]:
# read clusterPair.lik.genes.maxCore.c
maxCore_c_file = open('../test/comm2genes_maxCore-c.csv', 'r')
maxCore_c2genes = {}
for line in maxCore_c_file:    
    line = line.strip('\n').split('\t')    
    line = list(x for x in line if x != 'NA')
    (key, val) = [line[0].replace('Core', ''), line[1:]]
    maxCore_c2genes[key] = val
    
maxCore_c_file.close()   


In [26]:
maxCore_c2genes = dict((keys[key], value) for (key, value) in maxCore_c2genes.items())

In [27]:
plt.clf()
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'figure.figsize': (9, 8)})
#plt.style.use('dark_background')

# highlight filtered genes in clusterPair.lik.genes.maxCore.c
# Draw other edges including internal edges for unntotated genes and external edges
nx.draw_networkx_edges(
    G,
    pos=pos,    
    edgelist=G.edges() -internal,
    edge_color="silver",
    #edge_color = "#444444",
    alpha = 0.15,
    with_labels=False)

# Draw internal edges for anntotated gene modules
nx.draw_networkx_edges(
    G,
    pos=pos,    
    edgelist=internal,  
    edge_color = "#333333",
    alpha = 0.3,
    with_labels=False)

# Draw nodes
for i in list(comm2genes.keys())[:-1]: 
    nodes_i = comm2genes[i]
    nodes_i_in = maxCore_c2genes[i]
    nodes_i_out = set(nodes_i) - set(nodes_i_in)
    # draw nodes outside clusterPair.lik.genes.maxCore.c
    nx.draw_networkx_nodes(
        G,
        pos=pos,
        nodelist = nodes_i_out,
        node_size = 15,
        node_color=colormap[i],    
        alpha=0.4, 
        with_labels=False)
    nx.draw_networkx_nodes(
        G,
        pos=pos,
        nodelist = nodes_i_in,
        node_size = 50,
        node_color=colormap[i],
        label=list(keys.keys())[list(keys.values()).index(i)] + '_core_flt',
        alpha=0.7, 
        with_labels=False)
    
plt.legend(numpoints = 1) #Instead of numpoints = 1, use scatterpoints = 1
plt.savefig("communities_labeled_flt.pdf", dpi=300)